# Wykrywanie dna oka

In [ ]:
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import imageio
import cv2
import skimage.draw
from skimage import io, feature, color, feature, filters, draw
import math
import sys
import ipywidgets as widgets
from ipywidgets import interact
import os
np.set_printoptions(threshold=sys.maxsize)
from skimage.filters.edges import convolve
import skimage.morphology as mp
from skimage.morphology import square
from skimage.filters import frangi, hessian, sobel

from sklearn import metrics, ensemble
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
import functools
import operator
import pandas as pd

def img_green2gray(image):
    x=image.shape[0]
    y=image.shape[1]
    img = np.zeros( (x, y), dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if(image[i,j,1]<40):
                img[i,j]=0
            elif(sum(image[i,j])<100):
                img[i,j]=0
            else:
                img[i,j]=255-np.clip(image[i,j,1],0,140)
    return img

def img_green2gray_limit(image):
    x=image.shape[0]
    y=image.shape[1]
    img = np.zeros( (x, y), dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):       
            img[i,j]=np.clip(image[i,j,1],0,100)+np.sqrt(np.clip(image[i,j,1]-100,0,155))
    return img

def compare(image, image1):
    x=image.shape[0]
    y=image.shape[1]
    img = np.zeros( (x, y), dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if(image[i,j])<50:
                img[i,j]=0
            else:
                img[i,j]=np.clip(image[i,j]-image1[i,j], 40, 255)        
    return img

def compare_2(image, image1):
    x=image.shape[0]
    y=image.shape[1]
    img = np.zeros( (x, y), dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i,j]-image1[i,j]>0.85*image[i,j]:
                img[i,j]=255
            else:
                img[i,j]=0           
    return img

def compare_3(image, image1):
    x=image.shape[0]
    y=image.shape[1]
    img = np.zeros( (x, y), dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i,j]-image1[i,j]>0.85*image[i,j]:
                img[i,j]=255
            else:
                img[i,j]=0           
    return img


def apply_mask(img, mask):
    image = np.ndarray(shape=img.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if mask[i,j] == 255:
                image[i,j]=[0, 200, 100]
            else:
                image[i,j]=img[i,j]
    return image



def image_processing(image):
    img_green = image[:, :, 1]
    
    #mp.dilation(sobel(sobel(sobel(feature.canny(f_0, sigma=3))))
    
    img_pre_frangi = mp.dilation(sobel(feature.canny(img_green, sigma=0.5)))
    img_frangi = mp.dilation(sobel(feature.canny(img_green, sigma=0.5)))
    #img_frangi = frangi(img_green)
    for i in range(len(img_frangi)):
        for j in range(len(img_frangi[i])):
            if img_frangi[i][j] > 0.0000004:
                img_frangi[i][j] = 255
            else:
                img_frangi[i][j] = 0

    img_mask = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(img_mask, 1, 255, cv2.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    blackAndWhiteImage = cv2.erode(blackAndWhiteImage, kernel)

    final = img_frangi * blackAndWhiteImage
    
    kernel = np.ones((3, 3), np.uint8)
    arr = final > 0
    close = mp.remove_small_objects(arr, 10000)
    return close




In [ ]:

directory = 'images'
images = widgets.Dropdown(options=os.listdir(directory))

# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)
    
def show_images(file):
    img = imageio.imread(f'images/{file}').astype('uint8')
    plt.imshow(img)
_ = interact(show_images, file=images)

In [ ]:
img = imageio.imread(f'images/{images.value}').astype('uint8')
file2=images.value.rstrip('.ppm')+'.vk.ppm'
label = imageio.imread(f'labels/{file2}').astype('uint8')


f_0 =compare(img_green2gray(img),img_green2gray_limit(img))

#f_1 = mp.dilation(sobel(sobel(sobel(feature.canny(f_0, sigma=3)))))

f_1 = image_processing(img)


edge=sobel(feature.canny(img_green2gray(img), sigma=6))
edge=mp.dilation(edge,square(7))
mask= compare_2(f_1,edge)  
    

fig, ax = plt.subplots(ncols=2, figsize=(20, 30), nrows=3)

ax[0][0].imshow(img.astype('uint8'))
ax[0][0].set_title('Original image')    

ax[0][1].imshow(label, cmap=plt.cm.gray)
ax[0][1].set_title('Hand label image')

ax[1][0].imshow(f_0, cmap=plt.cm.gray)
ax[1][0].set_title('Image with filters')

ax[1][1].imshow(f_1, cmap=plt.cm.gray)
ax[1][1].set_title('Final image')

ax[2][0].imshow(edge, cmap=plt.cm.gray)
ax[2][0].set_title('Edge only')

ax[2][1].imshow(mask, cmap=plt.cm.gray)
ax[2][1].set_title('Final result')



## Ocena skuteczności

In [ ]:
tp=[]
tn=[]
fp=[]
fn=[]
masks=[]
for i in range(len(images.options)):
    print(images.options[i])
    tp.append(0)
    tn.append(0)
    fp.append(0)
    fn.append(0)
    img = imageio.imread(f'images/{images.options[i]}').astype('uint8')
    file2=images.options[i].rstrip('.ppm')+'.vk.ppm'
    label = imageio.imread(f'labels/{file2}').astype('uint8')

    f_1 = image_processing(img)
    edge=sobel(feature.canny(img_green2gray(img), sigma=6))
    edge=mp.dilation(edge,square(7))
    mask= compare_2(f_1,edge)
    masks.append(mask)
    for k in range(img.shape[0]):
        for j in range(img.shape[1]):
            if mask[k][j] == 255:
                if label[k][j] == 255:
                     tp[i]+=1
                else:
                     fp[i]+=1                 
            else:
                if label[k][j] == 255:
                     fn[i]+=1
                else:
                     tn[i]+=1 

In [ ]:
print(tp)
print(tn)
print(fp)
print(fn)

In [ ]:
accuracy=[]
sensitivity=[]
specificity=[]
for i in range(len(tp)):
    accuracy.append((tp[i]+tn[i])/(tp[i]+tn[i]+fp[i]+fn[i]))
    sensitivity.append(tp[i]/(tp[i]+fn[i]))
    specificity.append(tn[i]/(tn[i]+fp[i]))

In [ ]:
for i in range(len(images.options)):
    print(images.options[i])
    print("accuracy: " + str(accuracy[i]))
    print("sensitivity: " +str(sensitivity[i]))
    print("specificity: " + str(specificity[i])) 
    print("G-Mean: " + str(math.sqrt(sensitivity[i]*specificity[i])))
    print("Confusion matrix")
    print("tp  " + str(tp[i]) + "  fp  " + str(fp[i]) )
    print("fn  "  + str(fn[i]) +  "  tn  "  + str(tn[i]) + "\n")




In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=len(images.options)//2+(len(images.options)%2), figsize=(40, 10*len(images.options)//2+(len(images.options)%2)))
for i in range(len(images.options)):
    img = imageio.imread(f'images/{images.options[i]}').astype('uint8')
    ax[i//2][0+(i%2)*2].imshow(img.astype('uint8'))
    ax[i//2][0+(i%2)*2].set_title(f'Original image no. {images.options[i]}')
    img2=apply_mask(img, masks[i])
    ax[i//2][1+(i%2)*2].imshow(img2.astype('uint8'))
    ax[i//2][1+(i%2)*2].set_title(f'Original image no. {i} with mask ',)
fig.savefig('image_mask.png')

## Las decyzyjny 

In [ ]:
def normalize(img):
    return img / np.max(img)

def calc_contrast(val, lo, hi):
    if val < lo:
        return 0
    if val > hi:
        return 1
    return (val - lo)/(hi - lo)


def contrast_image(img, low, high):
    res = [[calc_contrast(x, low, high) for x in y] for y in img]
    return np.array(res)



def threshold(img, threshpoint):
    return np.array([[1 if x >= threshpoint else 0 for x in y] for y in img])


def apply_mask(img, mask):
    print(np.shape(img))
    print(np.shape(mask))
    res = []
    for i in range(len(img)):
        res.append([])
        for j in range(len(img[i])):
            res[i].append(img[i][j] if mask[i][j] == 0 else 1)
    return np.array(res)

def load_img(img_num, directory):
    if img_num < 10:
        img_num = f"im000{img_num}"
    elif img_num < 100:
        img_num = f"im00{img_num}"
    else:
        img_num = f"im0{img_num}"
    if directory == 'photos':
        return np.array(color.rgb2gray(io.imread(f'images/{img_num}.ppm')))
    elif directory == 'labels':
        return np.array(io.imread(f'labels/{img_num}.vk.ppm'))
    

img_num = 163
img = load_img(img_num, 'photos') #default picture
slice_size = 8
#plt.imshow(img)


In [ ]:
directory = 'images'
images = widgets.Dropdown(options=os.listdir(directory))
# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)
    
def show_images(file):
    img = imageio.imread(f'images/{file}').astype('uint8')
    plt.imshow(img)
_ = interact(show_images, file=images)

In [ ]:
#Prepare dataframe
def increase_contrast(img):
    nonzero_photo = img[np.nonzero(img)]
    percentiles = np.percentile(nonzero_photo,(2, 99))
    return contrast_image(img, percentiles[0], percentiles[1])

def detect_edges(img):
    img_sobel = filters.sobel(img)
    perc = np.percentile(img_sobel, (1, 99))
    return contrast_image(img_sobel, perc[0], perc[1])

def apply_morphology(img):
    result = threshold(img, 0.23)
    result = mp.erosion(result, mp.square(2))
    result = mp.dilation(result, mp.square(2))
    return mp.dilation(result, mp.square(2))

def flatten(arr):
    return functools.reduce(operator.iconcat, arr, [])

def print_evaluation_summary(labels, ground_truth):
    
    cf_mx = metrics.confusion_matrix(ground_truth, labels)
    print('Accuracy: ', (cf_mx[0,0]+cf_mx[1,1])/sum(flatten(cf_mx)))
    print('Sensitivity: ', cf_mx[1,1]/(cf_mx[1,0] + cf_mx[1,1]))
    print('Specificity: ', cf_mx[0,0]/(cf_mx[0,0] + cf_mx[0,1]))
    
    print("G-Mean: " + str(math.sqrt((cf_mx[1,1]/(cf_mx[1,0] + cf_mx[1,1]))*(cf_mx[0,0]/(cf_mx[0,0] + cf_mx[0,1])))))
    
    print("Confusion matrix")
    print("tp  " + str(cf_mx[1,1]) + "  fp  " + str(cf_mx[0,1]) )
    print("fn  "  + str(cf_mx[1,0]) +  "  tn  "  + str(cf_mx[1,1]) + "\n")    
    

def round_to_slice_size(dim):
    return slice_size * ((dim-slice_size) // slice_size)

def slice_image(image, step):
    slices = []

    for r in range(0, round_to_slice_size(image.shape[0]), step):
        for c in range(0, round_to_slice_size(image.shape[1]), step):
            slices.append(image[r:r+slice_size, c:c+slice_size])

    return slices

def to_hu_moments(imgs):
    hu_list = []
    for img in imgs:
        moments = cv2.moments(np.vectorize(float)(img))
        hu_moments = cv2.HuMoments(moments)
        hu_list.append(flatten(hu_moments))
        
    return pd.DataFrame(hu_list)

def reshape_labels(labels, shape):
    return labels.reshape(round_to_slice_size(shape[0]), round_to_slice_size(shape[1]))

def preprocess(img):
    img = normalize(img) 
    img = increase_contrast(img)
    img = detect_edges(img)
    return img

def take_samples(img, step):
    slices = slice_image(img, step)
    slices = [flatten(slice) for slice in slices]
    slices = np.append(slices, to_hu_moments(slices), axis=1)
    return pd.DataFrame(slices)

def label_samples(img, step):
    slices = slice_image(img, step)
    central_pixels = [np.round(slice[slice_size//2][slice_size//2]) for slice in slices]
    return pd.DataFrame(central_pixels)

def prepare_data(i, slice_step):
    x_img = load_img(i, 'photos')
    y_img = load_img(i, 'labels')
    x_img = preprocess(x_img)
    y_img = normalize(y_img)
    samples = take_samples(x_img, slice_step)
    labels = label_samples(y_img, slice_step)
    return samples.assign(label = labels)

def downsample(df):
    minority_count = int(np.sum(df.loc[:, 'label']))
    minority_df = df[df.label == 1]
    majority_df = df[df.label == 0]
    majority_df = resample(majority_df, replace=False, n_samples=minority_count)
    return pd.concat([minority_df, majority_df])

frames = []
training_images = [1,2,3,4,5,44,77,81,82]
for i in training_images:
    df = prepare_data(i, 6)
    df = downsample(df)
    frames.append(df)
training_df = pd.concat(frames)



In [ ]:
#train the classifier
clf = ensemble.RandomForestClassifier(n_estimators=100, random_state=1)
clf.fit(training_df.loc[:, training_df.columns != 'label'], training_df.loc[:,'label'])


scores = cross_val_score(clf, training_df.loc[:, training_df.columns != 'label'], training_df.loc[:,'label'], cv=5, verbose=20, n_jobs=4)
scores

In [ ]:
#classify the image
img_num = int(images.value.rstrip('.ppm').lstrip('im0')) #value from interactive box
full_img_df = prepare_data(img_num, 1)
labels = clf.predict(full_img_df.loc[:, full_img_df.columns != 'label'])

full_img_labels = normalize(load_img(img_num, 'labels'))
io.imshow(load_img(img_num, 'photos'))
io.show()
io.imshow(reshape_labels(labels, full_img_labels.shape))

In [ ]:
#output to files
clipped_output_img = full_img_labels[:round_to_slice_size(full_img_labels.shape[0]), :round_to_slice_size(full_img_labels.shape[1])]
print_evaluation_summary(labels, flatten(clipped_output_img))
io.imshow(full_img_labels)
labels_output = reshape_labels(labels, full_img_labels.shape)
shape = np.shape(labels_output)
res = apply_mask(load_img(img_num, 'photos')[:shape[0], :shape[1]],labels_output)
io.imshow(res)
cv2.imwrite('expert_mask.jpg', full_img_labels * 255)
cv2.imwrite('output_image.jpg', res * 255)
cv2.imwrite('our_mask.jpg', labels_output * 255)